In [1]:
import sys
import os # used for navigating to image path
import numpy as np
import pandas as pd
import csv
from PIL import Image
import imageio # used for writing images
from sklearn.model_selection import train_test_split
sys.path.append(os.path.abspath('../python'))
from helpers import *

In [2]:
SEED = 100
IMAGE_WIDTH_LIST = [189]#, 252, 336]
ALL_TILES = '../../data/tidy/conflict-split-images/all_tiles/'
PROCESSED_TILES_DIR = '../../data/tidy/conflict-split-images/preprocessed-tiles'
TILES_ASSIGNMENT = '../../data/tidy/conflict-split-images/tiles_assignment.csv'

In [3]:
d = pd.read_csv(TILES_ASSIGNMENT)

In [4]:
d.head()

,Filename,Classification
0,improbable-1_0_0.jpg,NaN
1,improbable-1_0_336.jpg,NaN
2,improbable-1_0_672.jpg,NaN
3,improbable-1_0_1008.jpg,NaN
4,improbable-1_0_1344.jpg,NaN


In [5]:
def assign_sample_tiles(unassigned_tiles_path):
    """Randomly classifies tiles as 'c'(conflict) or 'n'(no conflict)"""
    
    data = pd.read_csv(unassigned_tiles_path)
    data['Classification'] = np.random.choice(['c','n'], size=len(data))
    
    return(data)

In [6]:
sample_df = assign_sample_tiles(TILES_ASSIGNMENT)

In [7]:
sample_df.head()

,Filename,Classification
0,improbable-1_0_0.jpg,n
1,improbable-1_0_336.jpg,n
2,improbable-1_0_672.jpg,c
3,improbable-1_0_1008.jpg,c
4,improbable-1_0_1344.jpg,c


In [8]:
def generate_index(sample_data): #path_to_tiles_assignment
    """Serially labels all tiles by class and generates an index to data/tidy/conflict_split_images"""
    
#     data = pd.read_csv(path_to_tiles_assignment)
#     data = data.drop([0])
    
    df = pd.DataFrame(columns=['original', 'tile', 'classification'])
    
    conflict_counter    = 1
    no_conflict_counter = 1
    unknown_counter = 1
    counters = {}
        
    for row in sample_data.itertuples():
        tile = row.Filename
        original_image = str(os.path.splitext(tile)).split('_')[0][2:]
        classification = row.Classification

        if classification.lower() == 'c':        
            save_label = 'conflict' + '-' + str(conflict_counter) + '.jpg'
            df.loc[row.Index] = [original_image] + [tile] + [save_label]
            conflict_counter += 1
        elif classification.lower() == 'n':
            save_label = 'no_conflict' + '-' + str(no_conflict_counter) + '.jpg'
            df.loc[row.Index] = [original_image] + [tile] + [save_label]
            no_conflict_counter += 1
        else:
            save_label = 'unknown' + '-' + str(unknown_counter) + '.jpg'
            df.loc[row.Index] = [original_image] + [tile] + [save_label]
            unknown_counter += 1
    
    df.to_csv('../../data/tidy/conflict-split-images/tile_index_mapping.csv', encoding='utf-8', index=False)
    
    counters['conflict'] = conflict_counter - 1
    counters['no_conflict'] = no_conflict_counter - 1
    counters['unknown'] = unknown_counter -1 
    
    print('Number of conflict tiles recorded:', counters['conflict'])    
    print('Number of no-conflict tiles recorded:', counters['no_conflict'])
    print('Number of unknown tiles:', counters['unknown'])
        
    return df, counters

In [9]:
df, c = generate_index(sample_df)

Number of conflict tiles recorded: 303
Number of no-conflict tiles recorded: 264
Number of unknown tiles: 0


In [10]:
df

,original,tile,classification
0,improbable-1,improbable-1_0_0.jpg,no_conflict-1.jpg
1,improbable-1,improbable-1_0_336.jpg,no_conflict-2.jpg
2,improbable-1,improbable-1_0_672.jpg,conflict-1.jpg
3,improbable-1,improbable-1_0_1008.jpg,conflict-2.jpg
4,improbable-1,improbable-1_0_1344.jpg,conflict-3.jpg
...,...,...,...
562,probable-2,probable-2_3584_1344.jpg,conflict-299.jpg
563,probable-2,probable-2_3584_1680.jpg,conflict-300.jpg
564,probable-2,probable-2_3584_2016.jpg,conflict-301.jpg
565,probable-2,probable-2_3584_2352.jpg,conflict-302.jpg


In [11]:
def rename_tiles_conflict(data):
    for filename in os.listdir(ALL_TILES):
        for row in data.itertuples():
            tile = row.tile
            classification = row.classification
            if tile == filename:
                os.rename(ALL_TILES + filename, ALL_TILES + classification)
                continue

In [12]:
rename_tiles_conflict(df)

In [13]:
def getImageOneHotVector(image_file_name):
    """Returns one-hot vector encoding for each sub-image based on the image file name:
    Conflict: 1
    No conflict: 0
    Unknown: -1
    """
    word_label = image_file_name.split('-')[0]
    if word_label == 'conflict' : 
        return 1
    elif word_label == 'no_conflict': 
        return 0
    else:
        return -1 # if label is not present for current image

In [14]:
labels = []
for filename in os.listdir(ALL_TILES):
    labels.append(filename.split('-')[0])

In [15]:
print(set(labels))

{'no_conflict', 'conflict'}


In [16]:
def processImageData(image_width, seed_value, save_image_binary_files=True, test = False): # original size 4032 × 3024 px
    data_train = []
    data_test = []
    image_list = os.listdir(ALL_TILES) 
    random.seed(seed_value) #seed for repeatability
#     print("Preprocessing images for image width " + str(image_width) + "px")
    image_list_train, image_list_test =  train_test_split(image_list, test_size = .2, random_state = seed_value)

    for image_index in image_list:
        label = getImageOneHotVector(image_index)
        if label == -1: # if image labeled as unknown, move to next one
            continue
        path = os.path.join(ALL_TILES, image_index)
        image = Image.open(path) # read in image
#         print(np.array(image).shape)
        image_width, image_height = image.size
#         image_width = int(image_width) 
#         if rectangular==True:
#             image_height = getRectangularImageHeight(image_width) #int(image.size[0] * image_width/image.size[1]) ##because of input orientation, this is flipped.
#         else:
#             image_height = image_width
#         print("Image shape: " + str(image.size))            
        if test == True:
            pass
        if image_index in image_list_train:
            data_train.append([image_list_train, label])
        else:
            data_test.append([image_list_train, label])            
#     print(len(data_train))
#     print(len(data_test))  
    print("Training Images:", (np.array([x[1] for x in data_train])).sum(axis=0) )
    print("Test Images:", (np.array([x[1] for x in data_test])).sum(axis=0) )
    filename_prefix = 'w-' + str(image_width) + 'px-h-' + str(image_height)+ 'px'
    data_filename_train = filename_prefix+ "-train.npy"
    data_filename_test = filename_prefix + "-test.npy"
    if not os.path.exists(PROCESSED_TILES_DIR): # check if 'tidy/preprocessed_tiles' subdirectory does not exist
        os.makedirs(PROCESSED_TILES_DIR) # if not, create it    
    if save_image_binary_files == True:
        if test == True:
            data_filename_train = 'testing-' + data_filename_train
            data_filename_test = 'testing-' + data_filename_test
        np.save(os.path.join(PROCESSED_TILES_DIR, data_filename_train), data_train) #save as .npy (binary) file
        np.save(os.path.join(PROCESSED_TILES_DIR, data_filename_test), data_test) #save as .npy (binary) file        
        print("Saved " + data_filename_train + " to data/tidy/" + PROCESSED_TILES_DIR)
        print("Saved " + data_filename_test + " to data/tidy/" + PROCESSED_TILES_DIR)        
    return  #(image_selection_array, class_list)

In [17]:
for width in IMAGE_WIDTH_LIST:
    processImageData(width, seed_value=SEED, save_image_binary_files=True, test=False)

Training Images: 244
Test Images: 59
Saved w-336px-h-448px-train.npy to data/tidy/../../data/tidy/conflict-split-images/preprocessed-tiles
Saved w-336px-h-448px-test.npy to data/tidy/../../data/tidy/conflict-split-images/preprocessed-tiles


C:\Users\nasko\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)
